In [ ]:
!pip install opencv-python-headless fer

import cv2
from fer import FER
import csv
from datetime import datetime, timedelta
import collections

# Initialize FER emotion detector
emotion_detector = FER()

# Prompt for a unique user ID or name
user_id = input("Enter your user ID or name: ").strip()
filename = f"{user_id}_emotion_log.csv"  # Create a unique filename for each user

# Start video capture (from webcam)
cap = cv2.VideoCapture(0)

# Open a unique CSV file to save the log for the specific user
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Timestamp', 'Emotion'])  # Header row

    while True:
        # Track emotions for each 1-minute interval
        start_time = datetime.now()
        end_time = start_time + timedelta(minutes=1)
        minute_emotions = []

        # Collect emotions within the 1-minute interval
        while datetime.now() < end_time:
            # Capture frame-by-frame from the webcam
            ret, frame = cap.read()
            if not ret:
                break

            # Detect emotions in the frame
            emotions = emotion_detector.detect_emotions(frame)

            # Collect detected emotions for this 1-minute period
            for emotion_data in emotions:
                emotion = emotion_data['emotions']
                dominant_emotion = max(emotion, key=emotion.get)  # Find the dominant emotion
                minute_emotions.append(dominant_emotion)

            # Draw bounding boxes and captions on the frame in real-time
            for emotion_data in emotions:
                bbox = emotion_data['box']
                dominant_emotion = max(emotion_data['emotions'], key=emotion_data['emotions'].get)
                caption = f"Emotion: {dominant_emotion}"
                
                # Draw bounding box and emotion text
                cv2.rectangle(frame, (bbox[0], bbox[1]), 
                              (bbox[0] + bbox[2], bbox[1] + bbox[3]), 
                              (0, 255, 0), 2)
                cv2.putText(frame, caption, 
                            (bbox[0], bbox[1] - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            0.9, (36, 255, 12), 2)

            # Display the resulting frame with emotion captions
            cv2.imshow('Emotion Detection', frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                exit()

        # After 1 minute, determine the dominant emotion for the period
        if minute_emotions:
            overall_dominant_emotion = collections.Counter(minute_emotions).most_common(1)[0][0]
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M')
            print(f"Detected Emotion for Last 1 Minute: {overall_dominant_emotion}")
            
            # Ask for user confirmation
            user_response = input(f"Is '{overall_dominant_emotion}' the correct detected emotion? (yes/no): ").strip().lower()

            if user_response == 'yes':
                print(f"Final confirmed emotion: {overall_dominant_emotion}")
                writer.writerow([timestamp, overall_dominant_emotion])  # Log the confirmed emotion
            else:
                print("Retrying detection for another minute...")

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
